# Here I wrote a simple code for image augmentation along with their masks to increase training data for MASK-RCNN. This code works for annotations made in via annotator version 1.0.6. but modifying it for other versions should be straightforward

In [0]:
import cv2
from google.colab import files
import numpy as np
import json
import skimage
from skimage.io import imread, imshow, imread_collection, concatenate_images, imsave
from skimage.transform import resize
import os
import copy

In [0]:
uploaded = files.upload()
!unzip bacteria.zip # unzips with a folder bacteria with two subfolders "train" and "val" in it. 

In [0]:
ROOT_DIR = os.getcwd()
bacteria_dataset_path = os.path.join(ROOT_DIR, "bacteria")
dataset_dir = os.path.join(bacteria_dataset_path, "train") # change train to val and rerun this line and the cell below to get results for validation data

In [0]:
# Running this cell results in producing and saving augmented images and .json file for augmented masks on root directory.
annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
# five kinds of augmentation I'm implementing are flipping the image with respect to x and y axes, and rotating it 90, 180, 270 degrees.
# The reason I'm not performing rotation on intermidiate angles is that then I'll have resize the image and fill some space with zeros
# and that proved to be counter-productive for detection (at least in case of my high density low contrast images

# data structures to keep mask information for augmented images
flip_y_dict = {}
flip_x_dict = {}
rot_90_dict = {}
rot_180_dict = {}
rot_270_dict = {}

for key in annotations:
  a = annotations[key]
  image_path = os.path.join(dataset_dir, a['filename'])
  image = skimage.io.imread(image_path)
  height, width, depth = image.shape[:3]

  # since bacteria are axisymmetric, flipped images are justified for training.
  # making a new image called  flipped y
  flipped_y = image.copy()
  for i in range(height):
    for j in range(width):
      flipped_y[i,j]=image[height - 1 - i, j]
  flipped_y_name = "flip_y_"+ a['filename']
  imsave(flipped_y_name,flipped_y)

   # making a new image called flipped x
  flipped_x = image.copy()
  for i in range(height):
    for j in range(width):
      flipped_x[i,j]=image[i,width - 1 - j]
  flipped_x_name = "flip_x_"+ a['filename']
  imsave(flipped_x_name,flipped_x)

   # making a new image called rot 180 
  rot_180 = image.copy()
  for i in range(height):
    for j in range(width):
      rot_180[i,j]=image[height - 1 - i,width - 1 - j]
  rot_180_name = "rot_180_"+ a['filename']
  imsave(rot_180_name,rot_180)

   # making a new image called rot 270
  rot_270 = np.ndarray([width,height,depth],int)
  for i in range(width):
    for j in range(height):   
      rot_270[i,j]=image[height-1-j,i] #automatically assignes depth array values one to one
  rot_270_name = "rot_270_"+ a['filename']
  imsave(rot_270_name,rot_270)

   # making a new image called rot 90
  rot_90 = rot_270.copy()
  for i in range(width):
    for j in range(height):
      rot_90[i,j]=image[j,width-i-1]
  rot_90_name = "rot_90_"+ a['filename']
  imsave(rot_90_name,rot_90)

  #making new mask with flipped y    
  flipped_mask_y = copy.deepcopy(a)
  flipped_mask_y['filename']="flip_y_"+ a['filename']
  region = flipped_mask_y['regions']
  for num in region.keys():
    val = region[num]
    attributes = val['shape_attributes']
    y_points = attributes['all_points_y'];
    for i in range(len(y_points)):
      y_points[i] = height - 1 - y_points[i]
  flip_y_dict["flip_y"+key]=flipped_mask_y

   #making new mask with flipped x    
  flipped_mask_x = copy.deepcopy(a)
  flipped_mask_x['filename']="flip_x_"+ a['filename']
  region = flipped_mask_x['regions']
  for num in region.keys():
    val = region[num]
    attributes = val['shape_attributes']
    x_points = attributes['all_points_x'];
    for i in range(len(x_points)):
      x_points[i] = width - 1 - x_points[i]
  flip_x_dict["flip_x"+key]=flipped_mask_x

  #making new mask for 180 degree rotation    
  rot_mask_180 = copy.deepcopy(a)
  rot_mask_180['filename']="rot_180_"+ a['filename']
  region = rot_mask_180['regions']
  for num in region.keys():
    val = region[num]
    attributes = val['shape_attributes']
    x_points = attributes['all_points_x'];
    y_points = attributes['all_points_y'];
    for i in range(len(x_points)): #must be the same number as in y_points
      x_points[i] = width - 1 - x_points[i]
      y_points[i] = height - 1 - y_points[i]
  rot_180_dict["rot_180"+key]=rot_mask_180

  #making new mask for 90 degree rotation    
  rot_mask_90 = copy.deepcopy(a)
  rot_mask_90['filename']="rot_90_"+ a['filename']
  region = rot_mask_90['regions']
  for num in region.keys():
    val = region[num]
    attributes = val['shape_attributes']
    x_points = attributes['all_points_x'];
    y_points = attributes['all_points_y'];
    for i in range(len(x_points)): # must be the same number as in y_points
      temp = x_points[i]
      x_points[i] = y_points[i]
      y_points[i] = width - 1 - temp
  rot_90_dict["rot_90"+key]=rot_mask_90

   #making new mask for 90 degree rotation    
  rot_mask_270 = copy.deepcopy(a)
  rot_mask_270['filename']="rot_270_"+ a['filename']
  region = rot_mask_270['regions']
  for num in region.keys():
    val = region[num]
    attributes = val['shape_attributes']
    x_points = attributes['all_points_x'];
    y_points = attributes['all_points_y'];
    for i in range(len(x_points)): #must be the same number as in y_points
      temp = x_points[i]
      x_points[i] = height - 1 - y_points[i]
      y_points[i] = temp
  rot_270_dict["rot_270"+key]=rot_mask_270

for key in flip_y_dict:
  annotations[key] = flip_y_dict[key]
for key in flip_x_dict:
  annotations[key] = flip_x_dict[key]
for key in rot_90_dict:
  annotations[key] = rot_90_dict[key]
for key in rot_180_dict:
  annotations[key] = rot_180_dict[key]
for key in rot_270_dict:
  annotations[key] = rot_270_dict[key]

with open("bacteria.json", "w") as outfile: 
    json.dump(annotations, outfile) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: rot_270_im1.jpg is a low contrast image
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: rot_90_im1.jpg is a low contrast image
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: rot_270_im2.jpg is a low contrast image
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: rot_90_im2.jpg is a low contrast image
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: rot_270_im3.jpg is a low contrast image
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: UserWarning: rot_90_im3.jpg is a low contrast image
